# Setup

In [1]:
string_min = "23 64 5 4 64 3 64 2 24 64 64 64 64 17 19 64 64 64 25 64 9 26 64 1 64 64 13 64 64 16 64 0 64 64 8 6 64 10 18 64 64 64 64 7 14 64 20 64 64 64 11 64 64 64 15 64 64 64 12 22 21 64 27 28"
w= 104
k =6

string_min = "12 0 32 32 32 3 32 32 2 1 32 32 4 5 7 9 32 8 32 32 32 6 32 32 32 10 32 32 13 11 32 14"
w= 3
k =5


In [2]:
# Make into a list
opt_order = [int(x) for x in string_min.split(" ")]

In [3]:
# Make sure all rankings are unique
n_kmers = 2**k
new_arr = []
counter = 0
for i in opt_order:
    if i == n_kmers:
        new_arr.append(n_kmers-1-counter)
        counter+=1
    else:
        new_arr.append(i)

print(new_arr)


[12, 0, 31, 30, 29, 3, 28, 27, 2, 1, 26, 25, 4, 5, 7, 9, 24, 8, 23, 22, 21, 6, 20, 19, 18, 10, 17, 16, 13, 11, 15, 14]


# DP based charged context counting (2 versions)

In [4]:
def fast_wide(minw,maxw,k,order):
    table = [[0,0] for _ in range(2**k)]
    ##transition table of the deBrujin automaton
    size = 2**k - order.count(2**k)
    c0 = size
    ordr = order.copy()
    for kmer in range(2**k):  #completing ordr
        if ordr[kmer]==2**k:
            ordr[kmer] = c0
            c0 +=1
    for i in range(2**k):
        a = (i<<1) & (2**k-1) # 0-successing kmer for i
        table[ordr[i]] = [ordr[a], ordr[a+1]] # transition table (on ranks!) for order
    total = [0]*maxw

    for rank in range(size):
        old=[0]*2**k
        new=[0]*2**k
        old[rank]=1 #initialization for pref
        for i in range(maxw):    #DP for pref-gc
            for ii in range(rank,2**k):
                for iii in range(2):
                    new[table[ii][iii]] += old[ii]
                old[ii] = 0
            old, new = new, old
            if i>= minw-1:
                total[i] += sum(old[rank:])


        for i in range(rank):
            new[i] = 0 #cleaning
            old[i] = 0 #initialization for suff
        for i in range(rank,2**k):
            old[i] = 1 #initialization for suff
        for i in range(maxw):
            old[rank] = 0 # discard windows having kmer of current rank not as a suffix
            for ii in range(rank,2**k):
                for iii in range(2):
                    new[table[ii][iii]] += old[ii]
                old[ii] = 0
            old, new = new, old
            if i>= minw-1:
                total[i] += old[rank]
    return total[minw-1:]

In [5]:
def prefix_charged_dp_256(mask: int, kmer_to_add: int, w: int, k: int) -> int:
    n = 1 << k          # ≤ 64
    biggest_bit = 1 << (k - 1)

    old_arr = [0] * n
    new_arr = [0] * n
    old_arr[kmer_to_add] = 1

    for _ in range(w):
        for curr in range(n):
            # only propagate if k-mer is *not* masked
            if ((mask >> curr) & 1) == 0:
                prev = curr >> 1
                alt = prev | biggest_bit
                new_arr[curr] = old_arr[prev] + old_arr[alt]
            else:
                new_arr[curr] = 0

        total = sum(new_arr)
        if total == 0:
            return 0

        old_arr, new_arr = new_arr, old_arr
        for i in range(n):
            new_arr[i] = 0

    return sum(old_arr)


def suffix_charged_dp_256_v2(old_mask: int, kmer_to_add: int, w: int, k: int) -> int:
    n = 1 << k                 # ≤ 64
    biggest_bit = 1 << (k - 1)

    # array means how many (assuming last kmer which will be ahead is the kmer_to_add suffix) contexts are suffix charged
    old_arr = [1] * n
    new_arr = [0] * n

    # all branches starting from it are now suffix charged
    old_arr[kmer_to_add] = 0
    for kmer in range(n):
        if ((old_mask >> kmer) & 1) == 1:
            old_arr[kmer] = 0

    # first iteration is w=2 (w=1 is handled by initialization), we need to reach w,
    # since eventually we add 1 at the end and get w+1 -> need w-1 iterations
    for _ in range(1, w):
        for curr in range(n):
            if ((old_mask >> curr) & 1) == 0:
                prev = curr >> 1
                new_arr[curr] = old_arr[prev] + old_arr[prev | biggest_bit]
            else:
                new_arr[curr] = 0

        if new_arr[kmer_to_add] == 0:
            return 0
        else:
            new_arr[kmer_to_add] = 0

        old_arr, new_arr = new_arr, old_arr
        for i in range(n):
            new_arr[i] = 0

    prev_to_kmer_to_add_1 = kmer_to_add >> 1
    prev_to_kmer_to_add_2 = prev_to_kmer_to_add_1 | biggest_bit

    total = 0
    if prev_to_kmer_to_add_1 != kmer_to_add:
        total += old_arr[prev_to_kmer_to_add_1]
    if prev_to_kmer_to_add_2 != kmer_to_add:
        total += old_arr[prev_to_kmer_to_add_2]

    return total

def total_charged_dp_256(order, w,k) -> int:
    total = 0
    mask = 0
    for index in range(1 << k):
        # get the position of kmer with index index
        kmer_to_add = order.index(index)
        prefix = prefix_charged_dp_256(mask, kmer_to_add, w, k)
        suffix = suffix_charged_dp_256_v2(mask, kmer_to_add, w, k)
        total += prefix + suffix
        # add kmer to the mask
        mask |= (1 << kmer_to_add)
        #print(f" {kmer_to_add}\t(index {index} in order)\t {total}")
    return total


# Tests

In [6]:
results = total_charged_dp_256(new_arr, w, k)
print(results)

105


In [7]:
results = fast_wide(1,w,k,new_arr)
print(results[w-1])

105
